In [5]:
import sys
import os

In [10]:
curr_dir = sys.path[0]

src_dir = os.path.abspath(os.path.join(curr_dir, os.pardir))
main_dir = os.path.abspath(os.path.join(src_dir, os.pardir))
notebooks_dir = os.path.join(main_dir, 'notebooks')

In [11]:
print(src_dir)
print(main_dir)
print(notebooks_dir)

/home/baambriz/q2_code/src
/home/baambriz/q2_code
/home/baambriz/q2_code/notebooks


In [12]:
sys.path

['/home/baambriz/q2_code/src/analysis',
 '/opt/conda/lib/python39.zip',
 '/opt/conda/lib/python3.9',
 '/opt/conda/lib/python3.9/lib-dynload',
 '',
 '/home/baambriz/.local/lib/python3.9/site-packages',
 '/opt/conda/lib/python3.9/site-packages',
 '/opt/conda/lib/python3.9/site-packages/IPython/extensions',
 '/home/baambriz/.ipython']

TODO:  Using the checkpoints to load the models used and return output to be visualized

In [14]:
import numpy as np
import pandas as pd

# loading q1 results

### Fully Connected NN

In [30]:
fcnn_predictions = pd.read_csv(main_dir + '/fcnn_predictions.csv', index_col = False)

In [31]:
fcnn_predictions

,hbb_prediction,qcd prediction,hbb_label,qcd_label
0,0.442031,0.557969,0.0,1.0
1,0.780242,0.219758,0.0,1.0
2,0.803030,0.196970,1.0,0.0
3,0.734925,0.265075,1.0,0.0
4,0.998344,0.001656,1.0,0.0
...,...,...,...,...
6293,0.934704,0.065296,1.0,0.0
6294,0.638185,0.361815,0.0,1.0
6295,0.921765,0.078235,0.0,1.0
6296,0.721061,0.278939,0.0,1.0


### Deep Sets NN

In [27]:
dsnn_predictions = pd.read_csv(main_dir + '/dsnn_predictions.csv', index_col=False)

In [28]:
dsnn_predictions

,hbb_prediction,qcd prediction,hbb_label,qcd_label
0,0.521441,0.478559,0.0,1.0
1,0.578373,0.421627,0.0,1.0
2,0.862276,0.137724,1.0,0.0
3,0.724772,0.275228,1.0,0.0
4,0.887631,0.112369,1.0,0.0
...,...,...,...,...
6293,0.894830,0.105170,1.0,0.0
6294,0.508380,0.491620,0.0,1.0
6295,0.663219,0.336781,0.0,1.0
6296,0.619102,0.380898,0.0,1.0


# loading q2 results

### Graph NN 

In [20]:
graph_predictions = pd.read_csv(main_dir + '/GENConv_predictions.csv', index_col=False)

In [22]:
graph_predictions = graph_predictions.to_numpy()

In [23]:
graph_predictions

array([[ -8.443853 ],
       [-14.67983  ],
       [ -5.623162 ],
       ...,
       [-10.7989435],
       [-14.898462 ],
       [ -3.807295 ]])

## Evaluation / Comparison